# Summorize Pharma COUNT and create xlsx-Files

In [20]:
import pandas as pd
import mysql.connector
import sys
sys.path.insert(0, '../../data/lib/')
import consts

In [21]:
df_acc = pd.read_csv('../../data/3. transformation/0_accumulations_2019.csv', encoding='UTF-8')
df_rec = pd.read_csv('../../data/3. transformation/0_list_2019.csv', names=consts.fix_columns + [consts.column_export_information], skiprows=1)

In [22]:
# Sum Total. Do not use "total", otherwise its different than in the database (because some companies can not count. Seriously...)
df_rec = df_rec.drop(columns=['total'])

df_rec = df_rec.fillna(0)
df_rec['total'] = (df_rec['donations_grants'] + df_rec['sponsorship'] + df_rec['registration_fees']
                    + df_rec['travel_accommodation']+ df_rec['fees'] + df_rec['related_expenses'])




In [23]:
df_agg_count = df_acc[df_acc.type.isin(['hcp_count', 'hco_count'])].copy()
df_agg_count = df_agg_count.groupby('source').agg({'total': 'sum'}).reset_index()
df_agg_count = df_agg_count.rename(columns={'total': 'name'})
df_agg_count.head()

,source,name
0,abbvie,0.0
1,allergan,13.0
2,almirall,51.0
3,amgen,56.0
4,astellas,13.0


In [24]:
df_rec_count = df_rec.groupby('source').agg({'name': 'count'}).reset_index()
df_rec_count.head()

,source,name
0,abbvie,438
1,allergan,116
2,almirall,103
3,amgen,328
4,astellas,103


## Merge

In [25]:
df_count = pd.concat([df_agg_count, df_rec_count])
df_count = df_count.groupby('source').agg({'name': 'sum'})

## Get Sum

In [26]:
df_acc_sum = df_acc[df_acc.type.isin(['hcp_amount', 'hco_amount', 'rnd'])].copy()
df_acc_sum = df_acc_sum.groupby('source').agg({'total': 'sum'})

df_rec_sum = df_rec.groupby('source').agg({'total': 'sum'})

df_sum = pd.concat([df_acc_sum, df_rec_sum])
df_sum = df_sum.groupby('source').agg({'total': 'sum'}).reset_index()

## Join

In [27]:
df_joined = df_count.merge(df_sum, how='left', on='source')
df_joined = df_joined.rename(columns={'name': 'empfaenger', 'total': 'summe_zahlungen'})
df_joined['durchschnitt_pro_empfaenger'] = round(df_joined['summe_zahlungen'] / df_joined['empfaenger'])

df_joined = df_joined.sort_values('durchschnitt_pro_empfaenger', ascending=False)
df_joined.head()

,source,empfaenger,summe_zahlungen,durchschnitt_pro_empfaenger
12,bristol,207.0,12057979.02,58251.0
45,roche,358.0,18551873.32,51821.0
42,pfizer,416.0,16715103.00,40181.0
9,biogen,71.0,2679404.08,37738.0
5,astrazeneca,438.0,15960037.67,36438.0


In [29]:
#Write to Excel
writer = pd.ExcelWriter('../../data/99. analyzes/excel/pharmas/anzahl_empfaenger.xlsx', options={'encoding':'utf-8'})

df_joined.to_excel(writer, 'Empfänger', index=True)

writer.save()

In [68]:
df_rec.head()

,name,location,country,address,plz,uci,donations_grants,sponsorship,registration_fees,travel_accommodation,fees,related_expenses,total,type,source,_export_information
0,Ettore Accolla,Fribourg,CH,Chemin Des Pensionnats 2,NaN,NaN,NaN,NaN,552.0,622.0,NaN,NaN,1174.0,hcp,abbvie,NaN
1,Peter Ackle,Ehrendingen,CH,Höhtalmatt 3,NaN,NaN,NaN,NaN,380.0,477.0,NaN,NaN,857.0,hcp,abbvie,NaN
2,Abdou El Azizi Ali El Wafa,La Chaux-De-Fonds,CH,Avenue Léopold-Robert 16-18,NaN,NaN,NaN,NaN,482.0,759.0,1000.0,NaN,2241.0,hcp,abbvie,NaN
3,Michael Andor,Uster,CH,Uster West 11,NaN,NaN,NaN,NaN,NaN,NaN,800.0,NaN,800.0,hcp,abbvie,NaN
4,Anca-Teodora Antonino,Genolier,CH,Route Du Muids 3,NaN,NaN,NaN,NaN,304.0,1152.0,NaN,NaN,1456.0,hcp,abbvie,NaN


In [ ]:
-- ALT

## Connect

In [2]:
db_connection = mysql.connector.connect(
    host="localhost",
    user="root",
    passwd="",
    database = "pharmagelder"
)

## Load Data

In [34]:
"""
df_list_raw = pd.read_sql("""SELECT 
    count( DISTINCT tra_fk_recipient) as count,
    pha_name as pharma,
    pha_id as pharmaid,
    tra_year as year
  FROM transaction
  LEFT JOIN pharma ON tra_fk_pharma = pha_id
  LEFT JOIN transaction_category on tra_fk_transaction_category = trc_id
  LEFT JOIN recipient ON rec_id = tra_fk_recipient
  WHERE tra_year = 2019
  GROUP BY tra_fk_pharma
  """, con=db_connection)
"""

In [35]:
df_list_raw.head(20)

,count,pharma,pharmaid,year
0,433,AbbVie,1,2019
1,115,Allergan,4,2019
2,95,Almirall,5,2019
3,322,Amgen,6,2019
4,103,Astellas Pharma AG,7,2019
5,366,AstraZeneca,8,2019
6,84,Baxter AG,10,2019
7,837,Bayer AG,11,2019
8,75,Mylan Pharma GmbH (BGP Products),12,2019
9,71,Biogen,13,2019


## Export

In [4]:
def export_year(df_data, year):
    print(year)
    
    if year == 'all':
        df_part = df_data
    else:
        df_part = df_data[df_data.year == year]
    
    #Generate total
    df_pivot_total = df_part.pivot_table(index=['pharma', 'type'], columns='category', values='sum_value', fill_value=0, aggfunc='sum', margins=True, margins_name='total')
    df_pivot_total = df_pivot_total.fillna(0)
    df_pivot_total = df_pivot_total[['donations_grants', 'sponsorship', 'registration_fees', 'travel_accommodation', 'fees', 'related_expenses', 'rnd', 'total']]
    df_pivot_total = df_pivot_total.apply(lambda x: round(x, 2), axis=1)  
    
    #Generate detailed
    """
    df_pivot_detail = df_part.pivot_table(index=['pharma', 'declaration'], columns='category', values='sum_value', aggfunc='sum', margins=True, margins_name='total')
    df_pivot_detail = df_pivot_detail.fillna(0)
    df_pivot_detail = df_pivot_detail[['donations_grants', 'sponsorship', 'registration_fees', 'travel_accommodation', 'fees', 'related_expenses', 'rnd', 'total']]
    df_pivot_detail = df_pivot_detail.apply(lambda x: round(x, 2), axis=1)
    """
    #Write to Excel
    writer = pd.ExcelWriter('../../data/99. analyzes/excel/pharmas/nach_hco_hcp_rnd_%s.xlsx' % year, options={'encoding':'utf-8'})

    df_pivot_total.to_excel(writer, 'total %s' % year, index=True)

    writer.save()
    
    df_pivot_total.head()

In [ ]:
df_list = df_list_raw.copy()
df_accu = df_accu_raw.copy()

#Concat
df_data = pd.concat([df_list, df_accu], sort=False)

#Export all
export_year(df_data, 'all')

#Export years
for year in df_data['year'].unique():
    export_year(df_data, year)

all
2015
2016
2017
